In [67]:
import pandas as pd
import json

# Load the CSV data
#csv_file_path = 'c:/Users/domin/Downloads/construction_costs.csv'
csv_file_path = 'c:/Users/domin/Downloads/Project Data - BC.csv'

df = pd.read_csv(csv_file_path, keep_default_na=False)

# Convert the dataframe to JSON format
data = df.to_dict(orient='records')

# only what should be included in the sw
data = [v for v in data if v["In SW"] == "1"]

valid_units = ["t", "m2", "m3", "unit"]

# Organize data by authors and stage
authors_data = {}
all_authors = set()  # Use a set to collect unique author names
for record in data:
    authors = record['Author'].split(', ')
    for author in authors:
        author = author.strip()
        all_authors.add(author)  # Add author to the set
        if author not in authors_data:
            authors_data[author] = {}
        if record['Stage'] not in authors_data[author]:
            authors_data[author][record['Stage']] = []
        
        # Append the record to the appropriate author and stage
        new_record = {
            "id": record['id'],
            "material_category": record['Material cat'],
            "material": record['Material'],
            "approach": record['Approach'],
            "tool": record['Tool'],
            "description": record['Description'],
            "unit": record['Unit'],
            "rate": record['mh/unit'],
            "locality": record['Locality'],
            "tech_note": record['Tech note'],
            "author": author,
            "year": record['Year'],
            "pages": record['Pages']
        }
        
        if "Formula" in record and record["Formula"]:
            new_record["formula"] = record["Formula"]
            new_record["input1_name"] = record["Input1 name"]
            if ("Param1" in record and record["Param1"]):
                new_record["param1"] = record["Param1"]
            
        authors_data[author][record['Stage']].append(new_record);

        if "Conversions" in record and record["Conversions"]:
            #print("conversion " + record["Conversions"])
            new_conversions = {}
            conversions = record["Conversions"].replace("; ", ";").split(";")
            for conversion in conversions:
                value, unit = conversion.split(" ")
                to_unit, from_unit = unit.split("/")
                #print(from_unit + " " + to_unit)
                assert isinstance(from_unit, str)
                assert isinstance(to_unit, str)                
                assert from_unit in valid_units
                assert to_unit in valid_units
                new_conversions[to_unit] = float(value)
            new_record["conversions"] = new_conversions
        

# Save JSON files for each author
for author, stages in authors_data.items():
    file_name = f'{author.replace(" ", "_")}.json'
    with open(file_name, 'w') as f:
        json.dump({"technologies": stages}, f, indent=4)

# Save the list of authors to file-list.json
file_list = sorted(all_authors)  # Sort the authors list for consistency
with open('file-list.json', 'w') as f:
    json.dump({"file-list": file_list}, f, indent=4)

print("Conversion complete!")

Conversion complete!


In [69]:
output_folder = 'C:/Users/domin/OneDrive/Documents/_Real Documents/Škola/Bakalarka/Rebex server/Construction-energetics/data/technologies/new/'
# Save JSON files for each author
for author, stages in authors_data.items():
    #file_name = f'{output_folder + author.replace(" ", "_").replace(",", "")}.json'
    file_name = f'{output_folder + author.replace(" ", "_")}.json'
    with open(file_name, 'w') as f:
        json.dump({"technologies": stages}, f, indent=4)

# Save the list of authors to file-list.json
file_list = sorted(all_authors)  # Sort the authors list for consistency
with open(output_folder + 'file-list.json', 'w') as f:
    json.dump({"file-list": file_list}, f, indent=4)

print("Conversion complete!")

Conversion complete!
